In [111]:
!pip install -q snscrape 

In [112]:
import os 
import pandas as pd 
from os import listdir
from datetime import date, datetime, timedelta

In [113]:
def date_range(start, end):
    delta = end - start  # as timedelta
    days = [start + timedelta(days=i) for i in range(delta.days + 1)]
    return_days = [day.date() for day in days]
    return return_days

start_date = datetime(2022, 3, 12)
end_date = datetime(2022, 4, 24)
    
date_range_for_data = date_range(start_date, end_date)


def create_tuples(date_range_list): 
    return_list = []
    
    # example [(2021-10-31, 2021-11-1), (2021-11-1, 2021-11-2)]
    for i in range(len(date_range_list)): 
        if not(date_range_list[i] == date_range_list[len(date_range_list)-1]):
            return_list.append((date_range_list[i], date_range_list[i+1]))

    return return_list

date_range_tuple = create_tuples(date_range_for_data)

In [114]:
# Get tweets function with 
# inputs (max_results, search_term, from_date, end_date)
# For us the from and end date will be the same 
# it will output a csv in the current directory with date as the start of the file
# and the search term (query at the start as well) 
def get_tweets(tweet_count, text_query, since_date, until_date):
    # Using OS library to call CLI commands in Python
    os.system('snscrape --jsonl --max-results {} --since {} twitter-search "{} until:{}"> text-query-tweets.json'.format( tweet_count, since_date, text_query, until_date))

    # Reads the json generated from the CLI command above and creates a pandas dataframe
    tweets_df = pd.read_json('text-query-tweets.json', lines=True)
    tweets_df.to_csv(f'{since_date}_{text_query}_tweets.csv', sep=',', index=False)

# Do not run this anymore

In [6]:
tweet_count = 1000
text_query = "russia ukraine oil"

for date in date_range_tuple:
    get_tweets(tweet_count, text_query, date[0], date[1])

# Data Preprocessing 

#### Run this before you proceed with cleaning up and filtering the data 

#### final_dict - Is this dictionary that contains all the data according to dates as the key 

In [115]:
current_path = os.getcwd()

path_tweets = f'{current_path}/russia_ukraine_tweet/'

files = listdir(path_tweets)

#read files which end with csv 
# put them in a list 
#Input: all files in the directory list of files 
#Output: filter only the files 
def filter_csv(files):
    
    return_list = []
    
    for f in files: 
        if f.endswith('csv'):
            return_list.append(f)
    
    return return_list 

# Tested - Works well
files_list = filter_csv(files)
            
    
# write another function which makes a dictionary from a list of csv files 
# Prerequisite for the dictionary - filenames need to begin with dates 
# Key: Date Value: CSV 
# Input list of csv files
# Output : Dictionary - date and its constituent df 
def create_data_dictionary(files_list): 
    return_dict = {}
    files_with_no_data = [] 
    current_path = os.getcwd()
    data_path = f'{current_path}/russia_ukraine_tweet'
    for f in files_list:
        try:
            value = pd.read_csv(f'{data_path}/{f}')
            key = f[0:10]
            return_dict[key] = value
        except:
            files_with_no_data.append(f)
            continue 
            
    # to check if there are any files which are empty 
    # there is another way to filter and add it in this function itself but I have broken down 
    # for easier understanding. 
    return ((return_dict, files_with_no_data))

In [116]:
final_dict, files_with_no_data = create_data_dictionary(files_list)

# without days with no data 
final_days_of_data = len(final_dict.keys())

# input : dictionary , list of files 
# out : dictionary updated with file names but value is a string - 'no data'
def insert_null_val(dict_, files): 
    for i in files: 
        dict_[i[0:10]] = 'no value'
    
    return dict_

#with days with no data 
final_dict = insert_null_val(final_dict, files_with_no_data)

## Cleaning up each dataframe individually 

In [118]:
# TODO: 
# Take a sample 
# Filter the data 
# Create functions and test on sample df shown below
# source = [twitter for android, twitter for ios, twitter web app, news sources] 
# language = en (english tweets only)
# clean up tweets 

# Sample data
sample_df = final_dict['2022-03-14']

# filter dataframe using 'language' column
def filter_lang(dataframe):
    dataframe = dataframe.loc[dataframe['lang'] == 'en']
    return dataframe

# filter dataframe using 'sourceLabel' column
def filter_source(dataframe):
    dataframe.loc[dataframe['sourceLabel'].str.contains('iPad|iPhone|Mac'), 'sourceLabel'] = 'twitter for ios'
    dataframe.loc[dataframe['sourceLabel'].str.contains('Android'), 'sourceLabel'] = 'twitter for android'
    dataframe.loc[dataframe['sourceLabel'].str.contains('News'), 'sourceLabel'] = 'news sources'
    return dataframe

# apply both filter_lang and filter_source functions to a dataframe
def filtered_data(dataframe):
    dataframe = filter_lang(dataframe)
    dataframe = filter_source(dataframe)
    return dataframe

filtered_data(sample_df)

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,snscrape.modules.twitter.Tweet,https://twitter.com/SnipeCityBoys/status/15035...,2022-03-14 23:59:41+00:00,@linuxrebel @1984writer @HeraldOpEd @MarkHertl...,@linuxrebel @1984writer @HeraldOpEd @MarkHertl...,1503521276773687302,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,NaN,NaN,NaN,1.503520e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN
1,snscrape.modules.twitter.Tweet,https://twitter.com/romired7/status/1503520802...,2022-03-14 23:57:48+00:00,@AshaRangappa_ India as a sovereign nation wil...,@AshaRangappa_ India as a sovereign nation wil...,1503520802561728512,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,6,...,NaN,NaN,NaN,1.503364e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN
2,snscrape.modules.twitter.Tweet,https://twitter.com/pegleghokie/status/1503520...,2022-03-14 23:57:41+00:00,@lmelendez49 @BrettButler1111 The pipeline is ...,@lmelendez49 @BrettButler1111 The pipeline is ...,1503520773100736517,"{'_type': 'snscrape.modules.twitter.User', 'us...",4,0,1,...,NaN,NaN,NaN,1.503512e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN
3,snscrape.modules.twitter.Tweet,https://twitter.com/Magnum_442/status/15035207...,2022-03-14 23:57:39+00:00,Does anyone really believe Russia invading Ukr...,Does anyone really believe Russia invading Ukr...,1503520768679985152,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,snscrape.modules.twitter.Tweet,https://twitter.com/patrici60764612/status/150...,2022-03-14 23:57:37+00:00,Respite for drivers as oil price drops $10 a b...,Respite for drivers as oil price drops $10 a b...,1503520756940029954,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,snscrape.modules.twitter.Tweet,https://twitter.com/4n4lisis/status/1503379065...,2022-03-14 14:34:35+00:00,🛢️ 📉\n\nOil falls $5 on Russia-Ukraine talk ho...,🛢️ 📉\n\nOil falls $5 on Russia-Ukraine talk ho...,1503379065461424130,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Covid19', 'UkraineRussiaWar', 'UkraineUnderA...",NaN
996,snscrape.modules.twitter.Tweet,https://twitter.com/norraccmretep/status/15033...,2022-03-14 14:34:11+00:00,"China is a trading nation and it's interests,t...","China is a trading nation and it's interests,t...",1503378965540573185,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,"[{'_type': 'snscrape.modules.twitter.Gif', 'th...",NaN,NaN,NaN,NaN,NaN,{'_type': 'snscrape.modules.twitter.Coordinate...,"{'_type': 'snscrape.modules.twitter.Place', 'f...",NaN,NaN
997,snscrape.modules.twitter.Tweet,https://twitter.com/chitownhustler/status/1503...,2022-03-14 14:33:24+00:00,"“Oil falls $7 on Russia-Ukraine talk hopes, Ch...","“Oil falls $7 on Russia-Ukraine talk hopes, Ch...",1503378770295676934,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,snscrape.modules.twitter.Tweet,https://twitter.com/lacrosski/status/150337872...,2022-03-14 14:33:13+00:00,@nunyabiz222 @Active4Ever @WhiteHouse @PressSe...,@nunyabiz222 @Active4Ever @WhiteHouse @PressSe...,1503378721813770242,"{'_type': 'snscrape.modules.twitter.User', 'us...",2,0,0,...,NaN,NaN,NaN,1.501745e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN


In [119]:
# check whether a string is English
# True - String is English, False - String contains non-english characters (excluding emojis and special characters)
def isEnglish(value):
    return value.isascii()

# Extract rows with English content from a dataframe
# may use this function instead of filter_lang()
def filter_english(dataframe):
    for i, r in dataframe.iterrows():
        dataframe.loc[i, ['isEnglish']] = isEnglish(r['content'])
    
    dataframe = dataframe.loc[dataframe['isEnglish'] == True]
    return dataframe